<a href="https://colab.research.google.com/github/Joaoplims/NLP-Projects/blob/main/TP1_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import gensim
import requests
import urllib.request
import zipfile
import numpy as np
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity
from io import StringIO
from gensim.models import Word2Vec

# Caminho para o arquivo de analogias
caminhoAnalogias = '/content/drive/MyDrive/TPData/questions-words.txt'

# Baixar o corpus text8
corpus = api.load('text8')

# Converter o corpus para uma lista de palavras
text8Data = [d for d in corpus]
print(text8Data[0][:50]) # o "vocabulario" com as stop words

## Função para treinar o modelo
def train_model(corpus, size, window, sg, epochs):
    model = Word2Vec(corpus, vector_size=size, window=window, sg=sg, epochs=epochs)
    model.save(f'model_size{size}_window{window}_sg{sg}_epochs{epochs}.model')
    return model

# Carregar o modelo salvo
cbow1 = Word2Vec.load('/content/model_size100_window5_sg0_epochs5.model')
cbow2 = Word2Vec.load('/content/model_size200_window10_sg0_epochs10.model')
cbow3 = Word2Vec.load('/content/model_size300_window15_sg0_epochs15.model')
skipgram1 = Word2Vec.load('/content/model_size100_window5_sg1_epochs5.model')
skipgram2 = Word2Vec.load('/content/model_size200_window10_sg1_epochs10.model')
skipgram3 = Word2Vec.load('/content/model_size300_window15_sg1_epochs15.model')

# Treinar diversos modelos CBOW
#cbow1 = train_model(text8Data, size=100, window=5, sg=0, epochs=5)
#cbow2 = train_model(text8Data, size=200, window=10, sg=0, epochs=10)
#cbow3 = train_model(text8Data, size=300, window=15, sg=0, epochs=15)

# Treinar diversos modelos Skip-gram
#skipgram1 = train_model(text8Data, size=100, window=5, sg=1, epochs=5)
#skipgram2 = train_model(text8Data, size=200, window=10, sg=1, epochs=10)
#skipgram3 = train_model(text8Data, size=300, window=15, sg=1, epochs=15)

# Função para avaliar o modelo


# Função atualizada para avaliar o modelo
def evaluate_analogy(analogy_file, model):
    with open(analogy_file, 'r') as file:
        lines = file.readlines()

    distances = []
    for line in lines:
        words = line.split()
        if len(words) != 4:
            continue

        # Verificar se todas as palavras estão no vocabulário antes de continuar
        if all(word in model.wv for word in words):
            vec = model.wv[words[1]] + model.wv[words[0]] - model.wv[words[2]]
            expected = model.wv[words[3]]
            vec_norm = vec / np.linalg.norm(vec)
            expected_norm = expected / np.linalg.norm(expected)
            distance = cosine_similarity([vec_norm], [expected_norm])[0][0]
            distances.append(distance)

    # Retornar 0 se nenhuma analogia foi avaliada para evitar divisão por zero
    if not distances:
        return 0

    return sum(distances) / len(distances)

# Reavaliar os modelos com a função atualizada
avg_distance_cbow1 = evaluate_analogy(caminhoAnalogias, cbow1)
avg_distance_cbow2 = evaluate_analogy(caminhoAnalogias, cbow2)
avg_distance_cbow3 = evaluate_analogy(caminhoAnalogias, cbow3)

avg_distance_skipgram1 = evaluate_analogy(caminhoAnalogias, skipgram1)
avg_distance_skipgram2 = evaluate_analogy(caminhoAnalogias, skipgram2)
avg_distance_skipgram3 = evaluate_analogy(caminhoAnalogias, skipgram3)

print(f'Média das distâncias do cosseno CBOW1: {avg_distance_cbow1}')
print(f'Média das distâncias do cosseno CBOW2: {avg_distance_cbow2}')
print(f'Média das distâncias do cosseno CBOW3: {avg_distance_cbow3}')

print(f'Média das distâncias do cosseno Skip-gram1: {avg_distance_skipgram1}')
print(f'Média das distâncias do cosseno Skip-gram2: {avg_distance_skipgram2}')
print(f'Média das distâncias do cosseno Skip-gram3: {avg_distance_skipgram3}')


def evaluate_analogy_euclidean(analogy_file, model):
    with open(analogy_file, 'r') as file:
        lines = file.readlines()

    distances = []
    for line in lines:
        words = line.split()
        if len(words) != 4:
            continue

        # Verificar se todas as palavras estão no vocabulário antes de continuar
        if all(word in model.wv for word in words):
            vec = model.wv[words[1]] + model.wv[words[2]] - model.wv[words[0]]
            expected = model.wv[words[3]]
            # Calcular a distância euclidiana entre os vetores
            distance = np.linalg.norm(vec - expected)
            distances.append(distance)

    # Retornar 0 se nenhuma analogia foi avaliada para evitar divisão por zero
    if not distances:
        return 0

    return sum(distances) / len(distances)

# Avaliar os modelos usando a distância euclidiana
avg_distance_cbow1_euc = evaluate_analogy_euclidean(caminhoAnalogias, cbow1)
avg_distance_cbow2_euc = evaluate_analogy_euclidean(caminhoAnalogias, cbow2)
avg_distance_cbow3_euc = evaluate_analogy_euclidean(caminhoAnalogias, cbow3)

avg_distance_skipgram1_euc = evaluate_analogy_euclidean(caminhoAnalogias, skipgram1)
avg_distance_skipgram2_euc = evaluate_analogy_euclidean(caminhoAnalogias, skipgram2)
avg_distance_skipgram3_euc = evaluate_analogy_euclidean(caminhoAnalogias, skipgram3)

print(f'Média das distâncias euclidianas CBOW1: {avg_distance_cbow1_euc}')
print(f'Média das distâncias euclidianas CBOW2: {avg_distance_cbow2_euc}')
print(f'Média das distâncias euclidianas CBOW3: {avg_distance_cbow3_euc}')

print(f'Média das distâncias euclidianas Skip-gram1: {avg_distance_skipgram1_euc}')
print(f'Média das distâncias euclidianas Skip-gram2: {avg_distance_skipgram2_euc}')
print(f'Média das distâncias euclidianas Skip-gram3: {avg_distance_skipgram3_euc}')




['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']
Média das distâncias do cosseno CBOW1: -0.08581378147646788
Média das distâncias do cosseno CBOW2: -0.11542647546064864
Média das distâncias do cosseno CBOW3: -0.11098307686019057
Média das distâncias do cosseno Skip-gram1: 0.0866412625382548
Média das distâncias do cosseno Skip-gram2: -0.004158192581531785
Média das distâncias do cosseno Skip-gram3: -0.028993744556859134
Média das distâncias euclidianas CBOW1: 11.26022947789775
Média das distâncias euclidianas CBOW2: 21.5715124196576
Média das distâncias euclidianas CBOW3: 31.75072866443917
Média das

# Experimento com Word2Vec

## Introdução
Este experimento consistiu em treinar 6 modelos Word2Vec, sendo 3 utilizando o algoritmo CBOW e 3 utilizando o algoritmo Skip-gram. O objetivo foi explorar como diferentes hiperparâmetros afetam os resultados dos modelos.

## Configuração dos Modelos
### Modelos CBOW
1. **CBOW1**: `size=100, window=5, sg=0, epochs=5`
2. **CBOW2**: `size=200, window=10, sg=0, epochs=10`
3. **CBOW3**: `size=300, window=15, sg=0, epochs=15`

### Modelos Skip-gram
1. **Skip-gram1**: `size=100, window=5, sg=1, epochs=5`
2. **Skip-gram2**: `size=200, window=10, sg=1, epochs=10`
3. **Skip-gram3**: `size=300, window=15, sg=1, epochs=15`

## Método de Avaliação
Após o treinamento, utilizei duas funções de avaliação:
- **Similaridade através da distância do cosseno**
- **Similaridade através da distância euclidiana**

Cada função comparou os resultados dos modelos com as palavras existentes em um arquivo de analogias fornecido pelo professor. As funções retornaram a média das distâncias para todas as analogias calculadas.

## Resultados

### Avaliação por Distância do Cosseno
<b> Média das distâncias do cosseno CBOW1</b>: -0.08581378147646788 <br>
<b> Média das distâncias do cosseno CBOW2</b>: -0.11542647546064864 <br>
<b> Média das distâncias do cosseno CBOW3</b>: -0.11098307686019057 <br>
<b> Média das distâncias do cosseno Skip-gram1</b>: 0.0866412625382548 <br>
<b> Média das distâncias do cosseno Skip-gram2</b>: -0.004158192581531785 <br>
<b> Média das distâncias do cosseno Skip-gram3</b>: -0.028993744556859134 <br>

### Avaliação por Distância Euclidiana
<b> Média das distâncias euclidianas CBOW1 </b>: 11.26022947789775 <br>
<b> Média das distâncias euclidianas CBOW2 </b>: 21.5715124196576  <br>
<b> Média das distâncias euclidianas CBOW3 </b>: 31.75072866443917 <br>
<b> Média das distâncias euclidianas Skip-gram1 </b>: 3.189628786597677 <br>
<b> Média das distâncias euclidianas Skip-gram2 </b>: 4.4843674945780085 <br>
<b> Média das distâncias euclidianas Skip-gram3 </b>: 5.588138200657965 <br>

## Conclusões
Os resultados indicam que o modelo **Skip-gram1** obteve o melhor desempenho nas avaliações de similaridade tanto pela distância do cosseno quanto pela distância euclidiana. Os hiperparâmetros para este modelo foram:
- `vector_size=100`
- `window=5`
- `epochs=5`

Este modelo apresentou uma média das distâncias do cosseno de aproximadamente 0.086 e uma média das distâncias euclidianas de aproximadamente 3.189, indicando que capturou bem as relações semânticas nos dados de analogia.

## Reflexão Final
A escolha dos hiperparâmetros é crucial para o desempenho dos modelos de linguagem. Diferentes combinações podem resultar em variações significativas nos resultados. O algoritmo Skip-gram, com hiperparâmetros mais modestos, mostrou-se mais eficaz neste experimento.

Uma possivel justificativa para isso é que o uso de parâmetros mais modesto em um corpus relativamente pequeno com o do Text8 permite que o modelo crie uma representação mais generalizável e menos propensa a overfitting. Além disso, janelas de menores captura uma relação menos ruidosa, obtendo melhores resultados. Por fim, o algoritmo Skip-gram pode ter sido beneficiado ao capturar o contexto das palavras dentro dos parâmetros mais modestos escolhidos para o experimento
